### Relative Strength Index (RSI) 

(Source: https://github.com/tristcoil/RSI-computation-with-Python)

RSI is an indicator that we can use to measure if given asset is priced to high or too low. Here we will describe how to calculate RSI with Python and Pandas.

Calculation is as follows:

$$RSI_{n} = 100 - \frac{100}{1 + rs_{n}}$$
$$rs_{n} = \frac{gain_{avg_{n}}}{loss_{avg_{n}}}$$

where
- $gain_{avg}$ is average gain over time window (period)
- $loss_{avg}$ is average loss over time window (period)
- $n$ signifies computational step
 
 
Gains - datapoints that fit following condition: 
- when there was price increase compared to previous day, gain is equal to that price gain value, otherwise set to zero

Losses - datapoints that fit following condition: 
- when there was price decrease compared to previous day, loss is equal to that price loss value, otherwise set to zero

In our computations we expect losses and gains to have non negative values, so we can take for example absolute value of losses, multiply losses by minus one or take absolute value of the $rs_{n}$.


 
 
 
 
 
Now the question is how will we compute the average gain and loss.
see https://en.wikipedia.org/wiki/Relative_strength_index for available average options.

We will use Exponential Moving Average (EMA) with decay $\alpha = \frac{1}{period}$.
Period is typically set to 14 data points (in our case it means days, but for example on hourly chart it would mean 14 hours). 

 
After we figure out what kind of average we are going to use, the RSI computation is rather straightforward. 



When RSI values are:
- above 70, asset is considered overbought (overvalued)
- below 30, asset is considered oversold (undervalued)

Even better is to look at divergence between price and RSI values:
- bullish divergence: price is trending down, RSI values are increasing (possible Long entry)
- bearish divergence: price is th=rending up, RSI values are decreasing (possible Short entry)

Interesting might also be to look at datapoints around 20 and 80 RSI. Or even to investigate price correlation to RSI on weekly chart. 
Weekly timeframes should remove the noise, hence weekly RSI value around 20 or 30 might be theoretically actionable buy signal, especially with bullish divergence.



In [1]:
# installations: 
%pip install yfinance --upgrade --no-cache-dir
%pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# sample_data = [54.8, 56.8, 57.85, 59.85, 60.57, 61.1, 62.17, 60.6, 62.35, 62.15, 62.35, 61.45, 62.8, 61.37, 62.5, 62.57, 60.8, 59.37, 60.35, 62.35, 62.17, 62.55, 64.55, 64.37, 65.3, 64.42, 62.9, 61.6, 62.05, 60.05, 59.7, 60.9, 60.25, 58.27, 58.7, 57.72, 58.1, 58.2]
# print(sample_data)

In [3]:
from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

# ___library_import_statements___
import pandas as pd

# for pandas_datareader, otherwise it might have issues, sometimes there is some version mismatch
pd.core.common.is_list_like = pd.api.types.is_list_like

# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  

import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

#newest yahoo API 
import yfinance as yahoo_finance

#optional 
#yahoo_finance.pdr_override()

%matplotlib inline


Pick a ticker and a date range:

In [4]:
# ___variables___
ticker = 'BTC-USD'

start_time = datetime.datetime(2022, 1, 1)
end_time = datetime.datetime.now().date().isoformat() # today

Download the daily data from Yahoo Finance API

In [5]:
# yahoo gives only daily historical data
connected = False
while not connected:
    try:
        df = web.get_data_yahoo(ticker, start=start_time, end=end_time)
        connected = True
        print('connected to yahoo')
    except Exception as e:
        print("type error: " + str(e))
        time.sleep( 5 )
        pass   

# use numerical integer index instead of date    
df = df.reset_index()
display(df.head(5))

connected to yahoo


,Date,High,Low,Open,Close,Volume,Adj Close
0,2022-01-01,47827.312500,46288.484375,46311.746094,47686.812500,24582667004,47686.812500
1,2022-01-02,47881.406250,46856.937500,47680.925781,47345.218750,27951569547,47345.218750
2,2022-01-03,47510.726562,45835.964844,47343.542969,46458.117188,33071628362,46458.117188
3,2022-01-04,47406.546875,45752.464844,46458.851562,45897.574219,42494677905,45897.574219
4,2022-01-05,46929.046875,42798.222656,45899.359375,43569.003906,36851084859,43569.003906


#### Compute RSI

In [6]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

Calculate the RSI values using our function and add them to the dataframe.

In [7]:
df['RSI'] = computeRSI(df['Adj Close'], 14)

At the beginning of the dataframe there are no RSI values calculated, since we need to calculate them using sufficient time window (14 days).

In [8]:
display(df.head())

,Date,High,Low,Open,Close,Volume,Adj Close,RSI
0,2022-01-01,47827.312500,46288.484375,46311.746094,47686.812500,24582667004,47686.812500,NaN
1,2022-01-02,47881.406250,46856.937500,47680.925781,47345.218750,27951569547,47345.218750,NaN
2,2022-01-03,47510.726562,45835.964844,47343.542969,46458.117188,33071628362,46458.117188,NaN
3,2022-01-04,47406.546875,45752.464844,46458.851562,45897.574219,42494677905,45897.574219,NaN
4,2022-01-05,46929.046875,42798.222656,45899.359375,43569.003906,36851084859,43569.003906,NaN


In [9]:
display(df.tail())

,Date,High,Low,Open,Close,Volume,Adj Close,RSI
289,2022-10-17,19635.802734,19173.333984,19268.562500,19550.757812,27472552998,19550.757812,50.623054
290,2022-10-18,19666.994141,19144.769531,19550.466797,19334.416016,30580012344,19334.416016,47.669923
291,2022-10-19,19348.416016,19127.687500,19335.027344,19139.535156,22425387184,19139.535156,45.116721
292,2022-10-20,19315.199219,18971.458984,19138.085938,19053.740234,24493974420,19053.740234,43.999433
293,2022-10-21,19234.109375,18777.679688,19056.195312,19180.236328,32684976128,19180.236328,46.118137


#### Plotting

In [10]:
# plot price
plt.figure(figsize=(15,5))
plt.plot(df['Date'], df['Adj Close'])
plt.title('Price chart (Adj Close)')
plt.show()


# plot correspondingRSI values and significant levels
plt.figure(figsize=(15,5))
plt.title('RSI chart')
plt.plot(df['Date'], df['RSI'])

plt.axhline(0, linestyle='--', alpha=0.1)
plt.axhline(20, linestyle='--', alpha=0.5)
plt.axhline(30, linestyle='--')

plt.axhline(70, linestyle='--')
plt.axhline(80, linestyle='--', alpha=0.5)
plt.axhline(100, linestyle='--', alpha=0.1)
plt.show()

#### Github repository:
git clone https://github.com/tristcoil/RSI-computation-with-Python.git

#### Sources:
- https://www.youtube.com/watch?v=VX6JBxImapM
- http://www.andrewshamlet.net/2017/06/10/python-tutorial-rsi/
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

Other:
- https://www.youtube.com/watch?v=y0pil2LbMxU
- https://en.wikipedia.org/wiki/Relative_strength_index
- https://www.investopedia.com/terms/r/rsi.asp